In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('/arlequin/laicite_gd.csv', delimiter=";")
df['source'] = df['screen name']
dx = df[['text','source']]
dx2 = dx.dropna()

def split_string(string, max_length=4096):
    return [string[i:i + max_length] for i in range(0, len(string), max_length)]

dx2['text'] = dx2['text'].apply(split_string)
dx2 = dx2.explode('text').reset_index(drop=True)

sent = dx2['text'].tolist()
s = [str(x) for x in sent]


FileNotFoundError: [Errno 2] No such file or directory: '/home/me/Téléchargements/laicite_gd.csv'

In [ ]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
df['source'] = df['screen name']
dx = df[['text','source']]
dx2 = dx.dropna()

def split_string(string, max_length=4096):
    return [string[i:i + max_length] for i in range(0, len(string), max_length)]

dx2['text'] = dx2['text'].apply(split_string)
dx2 = dx2.explode('text').reset_index(drop=True)
sent = dx2['text'].tolist()
s = [str(x) for x in sent]

embeddings = model.encode(s, return_dense=True,max_length=4096)
embeddings = embeddings["dense_vecs"]
df1 = pd.DataFrame(embeddings)


In [ ]:
import genieclust

k20 = genieclust.Genie(n_clusters=500, cast_float32=True, gini_threshold=0.2, affinity="cosinesimil", exact=False, compute_all_cuts=True, compute_full_tree=True).fit_predict(embeddings)


h = k20[[20, 50, 100, 200, 500], :]
h = np.transpose(h)
h = pd.DataFrame(h, columns=['k20', 'k50', 'k100', 'k200', 'k500'])
h = h.drop_duplicates()


dx2['k20'] = k20[50, :]

s20 = dx2.groupby('k20').apply(lambda x: x.sample(n=7, replace=False) if len(x) >= 7 else x).reset_index(drop=True)
grouped_df = s20.groupby('k20')['text'].apply(' . '.join).reset_index()
grouped_df = grouped_df.rename(columns={'text': 'content'})



In [ ]:
from llama_cpp import Llama
from tqdm import tqdm

llm = Llama.from_pretrained(
    repo_id="failspy/Phi-3-mini-128k-instruct-abliterated-v3-GGUF",
    filename="Phi-3-mini-128k-instruct-abliterated-v3_q4.gguf",
    verbose=False, 
    n_ctx=4096,
    n_gpu_layers=-1,
)

def process_content(content):
    prompt = f"""<|system|>You are a helpful assistant.<|end|>
               <|user|>Summarize in french the following list of sentences in a short paragraph. Here is the list of sentence to summarize :{content}<|end|>
               <|assistant|>"""

    output = llm(prompt, max_tokens=2048, stop=["<|endoftext|>"])
    res = output["choices"][0]["text"].strip()
    return res

results = []
for content in tqdm(grouped_df['content'], desc="Summarizing"):  
    summary = process_content(content)
    results.append(summary)

dz = pd.DataFrame(results, columns=['Description'])

dz['k50'] = dz.index

m1 = pd.merge(h, dz, on='k50')
m1 = m1[['k20','k50','Description']]
m1 = m1.drop_duplicates()

gdf1 = m1.groupby('k20')['Description'].apply(' . '.join).reset_index()
gdf1 = gdf1.rename(columns={'text': 'content'})


results = []
for content in tqdm(gdf1['Description'], desc="Summarizing"):  
    summary = process_content(content)
    results.append(summary)

dz2 = pd.DataFrame(results, columns=['Description'])



dz.to_csv('dz1.csv', index=False)
dz2.to_csv('dz2.csv', index=False)
h.to_csv('h.csv', index=False)

"""
/tmp/ipykernel_64034/148662612.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dx2['text'] = dx2['text'].apply(split_string)
Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]
/tmp/ipykernel_64034/148662612.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dx2['text'] = dx2['text'].apply(split_string)
Inference Embeddings: 100%|██████████| 9718/9718 [02:25<00:00, 66.57it/s]
/home/me/ARLQ/lib/python3.10/site-packages/genieclust/genie.py:193: UserWarning: `compute_full_tree` is only available when `M` = 1 and `exact` is True
  warnings.warn("`compute_full_tree` is only available when `M` = 1 "
/tmp/ipykernel_64034/148662612.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  s20 = dx2.groupby('k20').apply(lambda x: x.sample(n=7, replace=False) if len(x) >= 7 else x).reset_index(drop=True)
Summarizing: 100%|██████████| 50/50 [12:30<00:00, 15.02s/it]
Summarizing: 100%|██████████| 20/20 [06:45<00:00, 20.28s/it]
"""


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/home/me/Téléchargements/laicite_gd.csv', delimiter=";")
df['source'] = df['screen name']
dx = df[['text','source']]
dx2 = dx.dropna()

def split_string(string, max_length=4096):
    return [string[i:i + max_length] for i in range(0, len(string), max_length)]

dx2['text'] = dx2['text'].apply(split_string)
dx2 = dx2.explode('text').reset_index(drop=True)

sent = dx2['text'].tolist()
s = [str(x) for x in sent]


In [1]:
from FlagEmbedding import BGEM3FlagModel

model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)
df['source'] = df['screen name']
dx = df[['text','source']]
dx2 = dx.dropna()

def split_string(string, max_length=4096):
    return [string[i:i + max_length] for i in range(0, len(string), max_length)]

dx2['text'] = dx2['text'].apply(split_string)
dx2 = dx2.explode('text').reset_index(drop=True)
sent = dx2['text'].tolist()
s = [str(x) for x in sent]

embeddings = model.encode(s, return_dense=True,max_length=4096)
embeddings = embeddings["dense_vecs"]
df1 = pd.DataFrame(embeddings)



ModuleNotFoundError: No module named 'FlagEmbedding'

In [2]:
import genieclust

k20 = genieclust.Genie(n_clusters=500, cast_float32=True, gini_threshold=0.2, affinity="cosinesimil", exact=False, compute_all_cuts=True, compute_full_tree=True).fit_predict(embeddings)


h = k20[[20, 50, 100, 200, 500], :]
h = np.transpose(h)
h = pd.DataFrame(h, columns=['k20', 'k50', 'k100', 'k200', 'k500'])
h = h.drop_duplicates()


dx2['k20'] = k20[50, :]

s20 = dx2.groupby('k20').apply(lambda x: x.sample(n=7, replace=False) if len(x) >= 7 else x).reset_index(drop=True)
grouped_df = s20.groupby('k20')['text'].apply(' . '.join).reset_index()
grouped_df = grouped_df.rename(columns={'text': 'content'})



ModuleNotFoundError: No module named 'genieclust'

In [ ]:
from llama_cpp import Llama
from tqdm import tqdm

llm = Llama.from_pretrained(
    repo_id="failspy/Phi-3-mini-128k-instruct-abliterated-v3-GGUF",
    filename="Phi-3-mini-128k-instruct-abliterated-v3_q4.gguf",
    verbose=False, 
    n_ctx=4096,
    n_gpu_layers=-1,
)

def process_content(content):
    prompt = f"""<|system|>You are a helpful assistant.<|end|>
               <|user|>Summarize in french the following list of sentences in a short paragraph. Here is the list of sentence to summarize :{content}<|end|>
               <|assistant|>"""

    output = llm(prompt, max_tokens=2048, stop=["<|endoftext|>"])
    res = output["choices"][0]["text"].strip()
    return res

results = []
for content in tqdm(grouped_df['content'], desc="Summarizing"):  
    summary = process_content(content)
    results.append(summary)

dz = pd.DataFrame(results, columns=['Description'])

dz['k50'] = dz.index

m1 = pd.merge(h, dz, on='k50')
m1 = m1[['k20','k50','Description']]
m1 = m1.drop_duplicates()

gdf1 = m1.groupby('k20')['Description'].apply(' . '.join).reset_index()
gdf1 = gdf1.rename(columns={'text': 'content'})


results = []
for content in tqdm(gdf1['Description'], desc="Summarizing"):  
    summary = process_content(content)
    results.append(summary)

dz2 = pd.DataFrame(results, columns=['Description'])



dz.to_csv('dz1.csv', index=False)
dz2.to_csv('dz2.csv', index=False)
h.to_csv('h.csv', index=False)